# Search by Retriever System with OpenSearch

## คำอธิบาย
Notebook นี้จะสาธิตการสร้างระบบค้นหาข้อมูลด้วย Retriever และ OpenSearch โดยใช้ Sentence Transformers สำหรับการแปลงข้อความเป็น vector embeddings และ OpenSearch เป็น vector database สำหรับการจัดเก็บและค้นหาข้อมูล

## การตั้งค่าพื้นฐาน
- OpenSearch Endpoint: http://34.101.178.186:9200
- Index Name: yourname_doc_index (กรุณาแก้ไขเป็นชื่อของคุณ)

## 1. การติดตั้ง Libraries ที่จำเป็น

ในขั้นตอนนี้เราจะติดตั้ง libraries ที่จำเป็นสำหรับการทำงานกับ OpenSearch และ Sentence Transformers

In [ ]:
# ติดตั้ง packages ที่จำเป็น
!pip install opensearch-py sentence-transformers datasets pandas numpy

## 2. Import Libraries และตั้งค่าเริ่มต้น

Import libraries ที่จำเป็นและกำหนดค่าการเชื่อมต่อกับ OpenSearch

In [ ]:
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from opensearchpy import OpenSearch, RequestsHttpConnection
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# กำหนดค่าการเชื่อมต่อ OpenSearch
OPENSEARCH_ENDPOINT = "http://34.101.178.186:9200"
OPENSEARCH_INDEX = "yourname_doc_index"  # ⚠️ CHANGE THIS TO YOUR NAME

print(f"OpenSearch Endpoint: {OPENSEARCH_ENDPOINT}")
print(f"Index Name: {OPENSEARCH_INDEX}")

## 3. สร้างการเชื่อมต่อกับ OpenSearch

สร้าง client สำหรับเชื่อมต่อกับ OpenSearch และทดสอบการเชื่อมต่อ

In [ ]:
# สร้าง OpenSearch client
try:
    # แยก URL เพื่อรับ host และ port
    url_parts = OPENSEARCH_ENDPOINT.replace('http://', '').replace('https://', '')
    host, port = url_parts.split(':')
    port = int(port)
    
    client = OpenSearch(
        hosts=[{'host': host, 'port': port}],
        http_compress=True,
        use_ssl=False,
        verify_certs=False,
        connection_class=RequestsHttpConnection,
        timeout=60
    )
    
    # ทดสอบการเชื่อมต่อ
    info = client.info()
    print("✅ เชื่อมต่อ OpenSearch สำเร็จ!")
    print(f"OpenSearch Version: {info['version']['number']}")
    
except Exception as e:
    print(f"❌ เชื่อมต่อ OpenSearch ไม่สำเร็จ: {e}")

## 4. โหลด Sentence Transformer Model

โหลดโมเดล Sentence Transformer สำหรับการแปลงข้อความเป็น vector embeddings

In [ ]:
# โหลด Sentence Transformer model
print("กำลังโหลด Sentence Transformer model...")
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# ทดสอบโมเดล
test_text = "ทดสอบการทำงานของโมเดล"
test_embedding = model.encode(test_text)

print(f"✅ โหลดโมเดลสำเร็จ!")
print(f"ขนาดของ embedding vector: {len(test_embedding)}")
print(f"ตัวอย่าง embedding (5 ตัวแรก): {test_embedding[:5]}")

## 5. สร้าง Index Schema สำหรับ OpenSearch

กำหนดโครงสร้างของ index ที่จะใช้เก็บข้อมูลและ vector embeddings

In [ ]:
# กำหนด mapping สำหรับ index
index_mapping = {
    "mappings": {
        "properties": {
            "id": {
                "type": "keyword"
            },
            "title": {
                "type": "text",
                "analyzer": "standard"
            },
            "content": {
                "type": "text",
                "analyzer": "standard"
            },
            "category": {
                "type": "keyword"
            },
            "embedding": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "created_at": {
                "type": "date"
            }
        }
    },
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}

# ลบ index เก่าถ้ามี
if client.indices.exists(index=OPENSEARCH_INDEX):
    client.indices.delete(index=OPENSEARCH_INDEX)
    print(f"ลบ index เก่า '{OPENSEARCH_INDEX}' แล้ว")

# สร้าง index ใหม่
response = client.indices.create(
    index=OPENSEARCH_INDEX,
    body=index_mapping
)

print(f"✅ สร้าง index '{OPENSEARCH_INDEX}' สำเร็จ!")
print(json.dumps(response, indent=2, ensure_ascii=False))

## 6. เตรียมข้อมูลตัวอย่าง

สร้างข้อมูลตัวอย่างสำหรับทดสอบระบบค้นหา

In [ ]:
# ข้อมูลตัวอย่าง
sample_documents = [
    {
        "id": "doc_001",
        "title": "การเรียนรู้ของเครื่อง (Machine Learning)",
        "content": "การเรียนรู้ของเครื่อง (Machine Learning) เป็นสาขาหนึ่งของปัญญาประดิษฐ์ที่เน้นการพัฒนาอัลกอริทึมและโมเดลทางสถิติที่คอมพิวเตอร์สามารถใช้เพื่อปฏิบัติงานเฉพาะได้อย่างมีประสิทธิภาพโดยไม่ต้องใช้คำสั่งที่ชัดเจน",
        "category": "AI/ML",
        "created_at": "2024-01-15T10:00:00Z"
    },
    {
        "id": "doc_002",
        "title": "การประมวลผลภาษาธรรมชาติ (Natural Language Processing)",
        "content": "การประมวลผลภาษาธรรมชาติ (NLP) เป็นสาขาของปัญญาประดิษฐ์ที่เกี่ยวข้องกับการปฏิสัมพันธ์ระหว่างคอมพิวเตอร์และภาษามนุษย์ รวมถึงการสอนคอมพิวเตอร์ให้เข้าใจ ตีความ และสร้างภาษามนุษย์อย่างมีความหมาย",
        "category": "AI/ML",
        "created_at": "2024-01-16T14:30:00Z"
    },
    {
        "id": "doc_003",
        "title": "ระบบจัดการฐานข้อมูล (Database Management System)",
        "content": "ระบบจัดการฐานข้อมูล (DBMS) เป็นซอฟต์แวร์ที่ช่วยในการจัดเก็บ จัดการ และเรียกใช้ข้อมูลจากฐานข้อมูล มีหน้าที่ในการควบคุมการเข้าถึงข้อมูล รักษาความปลอดภัย และรับประกันความสมบูรณ์ของข้อมูล",
        "category": "Database",
        "created_at": "2024-01-17T09:15:00Z"
    },
    {
        "id": "doc_004",
        "title": "การพัฒนาเว็บแอปพลิเคชัน (Web Application Development)",
        "content": "การพัฒนาเว็บแอปพลิเคชันเป็นกระบวนการสร้างแอปพลิเคชันที่ทำงานบนเว็บเบราว์เซอร์ ซึ่งประกอบด้วยการพัฒนาส่วน Frontend (หน้าบ้าน) และ Backend (หลังบ้าน) รวมถึงการจัดการฐานข้อมูลและ API",
        "category": "Web Development",
        "created_at": "2024-01-18T16:45:00Z"
    },
    {
        "id": "doc_005",
        "title": "ความปลอดภัยทางไซเบอร์ (Cybersecurity)",
        "content": "ความปลอดภัยทางไซเบอร์เป็นการปฏิบัติในการปกป้องระบบ เครือข่าย และโปรแกรมจากการโจมตีทางดิจิทัล การโจมตีเหล่านี้มักมีเป้าหมายเพื่อเข้าถึง เปลี่ยนแปลง หรือทำลายข้อมูลที่ละเอียดอ่อน",
        "category": "Security",
        "created_at": "2024-01-19T11:20:00Z"
    }
]

print(f"เตรียมข้อมูลตัวอย่าง {len(sample_documents)} documents")
for doc in sample_documents:
    print(f"- {doc['title']} ({doc['category']})")

## 7. สร้าง Embeddings และ Index ข้อมูล

แปลงข้อมูลเป็น vector embeddings และเก็บลงใน OpenSearch

In [ ]:
def create_document_embedding(title: str, content: str) -> List[float]:
    """
    สร้าง embedding จากชื่อเรื่องและเนื้อหา
    """
    # รวมชื่อเรื่องและเนื้อหาเป็นข้อความเดียว
    combined_text = f"{title} {content}"
    
    # สร้าง embedding
    embedding = model.encode(combined_text)
    
    return embedding.tolist()

# สร้าง embeddings และ index ข้อมูล
print("กำลังสร้าง embeddings และ index ข้อมูล...")

for doc in sample_documents:
    # สร้าง embedding
    embedding = create_document_embedding(doc['title'], doc['content'])
    
    # เตรียม document สำหรับ index
    doc_with_embedding = {
        **doc,
        'embedding': embedding
    }
    
    # Index document
    response = client.index(
        index=OPENSEARCH_INDEX,
        id=doc['id'],
        body=doc_with_embedding
    )
    
    print(f"✅ Indexed: {doc['title']} (Result: {response['result']})")

# รอให้ index refresh
client.indices.refresh(index=OPENSEARCH_INDEX)
print("\n🎉 Index ข้อมูลเสร็จสิ้น!")

## 8. สรุปและการใช้งานต่อ

ระบบค้นหาด้วย OpenSearch และ Sentence Transformers พร้อมใช้งานแล้ว!

### การใช้งานต่อไป:
1. เพิ่มข้อมูลเอกสารใหม่ๆ
2. ปรับปรุงการค้นหาให้แม่นยำขึ้น
3. สร้าง web interface
4. เพิ่มฟีเจอร์การกรองข้อมูล

In [ ]:
print("🎉 ระบบ Search by Retriever with OpenSearch เสร็จสมบูรณ์!")
print("\n📝 ขั้นตอนถัดไป:")
print("1. ทดสอบการค้นหาด้วยคำสำคัญต่างๆ")
print("2. เพิ่มเอกสารใหม่เข้าสู่ระบบ")
print("3. ปรับแต่งพารามิเตอร์การค้นหา")
print("4. สร้าง API หรือ web interface")
print("\n✨ ระบบพร้อมใช้งาน! ✨")